In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
# import seaborn as sns
from collections import OrderedDict

# Read from accompanying data_tools.py script contained in same working directory
import data_tools as tools

### Specify the CSV file containing the data to analyze


In [ ]:
in_csv = input("Full path to TAP-exported .csv file: ")

### Read in the data

In [ ]:
data = pd.read_csv(in_csv, parse_dates=['dates'], infer_datetime_format=True).drop(columns='Unnamed: 0')

### Show a preview of the data frame

In [ ]:
data.head()

### Demo for pulling all data annually between June and August with a cloud-filtering mask applied

In [ ]:
tools.mask(tools.dates(data, (dt.datetime(1985, 6, 1), dt.datetime(1985,8, 31))))

### Get Mean Annual MSAVI for June-July-August

#### This cell applies the filters demonstrated above and creates a new data frame

In [ ]:
params = {'start_mon': 6,
          
          'start_day': 1,
          
          'end_mon': 8,
         
          'end_day': 31,
         
          'field': 'msavi',
         
          'mask_field': 'qa'}

msavi = tools.get_seasonal_info(data, params)

### Change the params 'field' value to get additional indices (if available from plotting)

In [ ]:
# params = {'start_mon': 6,
#          'start_d': 1,
#          'end_mon': 8,
#          'end_d': 31,
#          'field': 'ndvi',
#          'mask_field': 'qa'}

# ndvi = tools.get_seasonal_info(data, params)

In [ ]:
# params = {'start_mon': 6,
#          'start_d': 1,
#          'end_mon': 8,
#          'end_d': 31,
#          'field': 'evi',
#          'mask_field': 'qa'}

# evi = tools.get_seasonal_info(data, params)

In [ ]:
# params = {'start_mon': 6,
#          'start_d': 1,
#          'end_mon': 8,
#          'end_d': 31,
#          'field': 'savi',
#          'mask_field': 'qa'}

# savi = tools.get_seasonal_info(data, params)

In [ ]:
plt.style.use('ggplot')

fig = plt.figure(figsize=(10,4), dpi=100)

ax = fig.add_subplot(111)

plt.xticks(tools.years(data), rotation=45, fontsize=8)

plt.xlim(tools.years(data)[0], tools.years(data)[-1])

# l = ax.plot(*plot_data(ndvi, 'max'), alpha=0.0)



# ax.fill_between(*plot_data(ndvi, 'max'), interpolate=True, color='darkgreen', alpha=0.7, label='NDVI')

# ax.fill_between(*plot_data(evi, 'max'), interpolate=True, color='c', alpha=0.7, label='EVI')

# ax.fill_between(*plot_data(savi, 'max'), interpolate=True, color='orange', alpha=0.7, label='SAVI')

ax.fill_between(*tools.plot_data(msavi, 'max'), interpolate=True, color='coral', alpha=0.7, label='MSAVI')

plt.suptitle("Maximum Annual Vegetation Indices for Summer Growing Months (June 1 - Aug 31)")

plt.legend(loc=(1, .7))

plt.show()

In [ ]:
plt.style.use('ggplot')

fig = plt.figure(figsize=(10,4), dpi=100)

ax = fig.add_subplot(111)

plt.xlim(tools.years(data)[0], tools.years(data)[-1])

plt.xticks(tools.years(data), rotation=45, fontsize=8)

# plt.plot(*plot_data(evi), '-o', ms=4, color='darkgreen')


# ax.fill_between(*tools.plot_data(ndvi, 'mean'), interpolate=True, color='darkgreen', alpha=0.7, label='NDVI')

# ax.fill_between(*tools.plot_data(evi, 'mean'), interpolate=True, color='c', alpha=0.7, label='EVI')

# ax.fill_between(*tools.plot_data(savi, 'mean'), interpolate=True, color='orange', alpha=0.7, label='SAVI')

ax.fill_between(*tools.plot_data(msavi, 'mean'), interpolate=True, color='coral', alpha=0.7, label='MSAVI')
plt.legend(loc=(1, 0.7))

plt.suptitle("Mean Annual Vegetation Indices for Summer Growing Months")


### Get the spectral signature for a target land cover

In [ ]:
in_csv = r'C:\Users\dzelenak\lcmap_tap\20180911-145646\20180911-145857_-602325_2323485.csv'

In [ ]:
data = pd.read_csv(in_csv, parse_dates=['dates'], infer_datetime_format=True).drop(columns='Unnamed: 0')

In [ ]:
data.head()

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'blues',
         'mask_field': 'qa'}

blues = tools.get_seasonal_info(data, params)

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'greens',
         'mask_field': 'qa'}

greens = tools.get_seasonal_info(data, params)

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'reds',
         'mask_field': 'qa'}

reds = tools.get_seasonal_info(data, params)

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'nirs',
         'mask_field': 'qa'}

nirs = tools.get_seasonal_info(data, params)

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'swir1s',
         'mask_field': 'qa'}

swir1s = tools.get_seasonal_info(data, params)

In [ ]:
params = {'start_mon': 6,
         'start_d': 1,
         'end_mon': 8,
         'end_d': 31,
         'field': 'swir2s',
         'mask_field': 'qa'}

swir2s = tools.get_seasonal_info(data, params)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

ax.plot(*plot_data(blues, 'mean'), 'blue', label='Blue')
ax.plot(*plot_data(greens, 'mean'), 'green', label='Green')
ax.plot(*plot_data(reds, 'mean'), 'red', label='Red')
ax.plot(*plot_data(nirs, 'mean'), 'darkred', label='NIR')
ax.plot(*plot_data(swir1s, 'mean'), 'orange', label='SWIR-1')
ax.plot(*plot_data(swir2s, 'mean'), 'purple', label='SWIR-2')



plt.suptitle('Annual Mean Spectral Signature for Summer Growing Months')

plt.legend(loc=(1, 0.55))



In [ ]:
plt.style.use('ggplot')

fig, ax1 = plt.subplots()

fig.suptitle("Mean Annual Vegetation Indices and Surface Reflectance for Summer Growing Months", x=0.5, y=0.9)

fig.set_size_inches((14, 8))

plt.xticks(years(data), rotation=45, fontsize=8)

plt.xlim(years(data)[0], years(data)[-1] + 1)

ax1.fill_between(*plot_data(ndvi, 'mean'), interpolate=True, color='darkgreen', alpha=0.7, label='NDVI')

ax1.fill_between(*plot_data(evi, 'mean'), interpolate=True, color='c', alpha=0.7, label='EVI')

ax1.fill_between(*plot_data(savi, 'mean'), interpolate=True, color='orange', alpha=0.7, label='SAVI')

ax1.fill_between(*plot_data(msavi, 'mean'), interpolate=True, color='coral', alpha=0.7, label='MSAVI')

ax1.set_xlabel('Year')

ax1.set_ylabel('Index Value', color='g')
ax1.tick_params('y', colors='g')

plt.legend(loc='upper left', bbox_to_anchor=(0, 1), framealpha=0.6)

plt.style.use('default')

ax2 = ax1.twinx()

ax2.plot(*plot_data(blues, 'mean'), 'blue', label='Blue', marker='^')
ax2.plot(*plot_data(greens, 'mean'), 'green', label='Green', marker='v')
ax2.plot(*plot_data(reds, 'mean'), 'red', label='Red', marker='D', ms=6)
ax2.plot(*plot_data(nirs, 'mean'), 'darkred', label='NIR', marker='s')
ax2.plot(*plot_data(swir1s, 'mean'), 'orange', label='SWIR-1', marker='d', ms=6)
ax2.plot(*plot_data(swir2s, 'mean'), 'purple', label='SWIR-2', marker='o')

ax2.set_ylabel('Reflectance', color='b')
ax2.tick_params('y', colors='b')

plt.legend(loc='upper right', bbox_to_anchor=(1, 1), framealpha=0.6)

plt.show()

In [ ]:
_ndvi = pd.DataFrame(ndvi).transpose()

In [ ]:
_ndvi['years'] = years(data)

In [ ]:
_ndvi.columns

In [ ]:
sns.violinplot(data=_ndvi.drop(columns=['years']))